# Example of next_pass application to Hurricane Milton, Florida, October 2024

## Background
* Knowing the time of a satellite overpass (OP) at a precise location is crucial to plan and prepare disaster impact studies. 
The script below can be used to predict the overpasses of the Landsat 8 & 9 and Sentinel 1 & 2 satellites over a  selected location. For Landsat 8 this occurs every 16 days and for Sentinel 2A / 2B this occurs every 10 days.

* The code calls the Python package 'next_pass' located at https://github.com/ehavazli/next_pass. The latter predicts the next overpass of the satellite of interest by scanning the relevant acquisition plans:
	- Landsat acquisition plans (json files) : https://landsat.usgs.gov/sites/default/files/landsat_acq/assets/json/cycles_full.json
	- Sentinel acquisition plans (KML files to import to Google Earth Pro) : https://sentinel.esa.int/web/sentinel/copernicus/sentinel-1/acquisition-plans
      

* This specific notebook showcases the application of the next_pass tool to hurricane Milton in Florida between October 3rd and October 11th, 2024 (https://maps.disasters.nasa.gov/arcgis/apps/MinimalGallery/index.html?appid=081982abfd5f4732b45df6db9f6de35e). The user can set the location using the "MiltonHurricaneFlorida" kml file 

## Tool Description

All what a user needs to provide is the precise location for which he desires to identify the next overpasses. The location can be input as a single point(latitude, longitude), or bounding box (SNEW coordinates) or a location file path (.kml). The script returns the next collect for Sentinel-1 and Sentinel-2 and the next passes, in ascending and descending directions separately, for Landsat-8 and Landsat-9.

- Specify a location 
- Run find_next_overpass for Sentinel 1 & 2 and the Landsats 8 & 9 
- Visualize the predicted overpasses

The outputs of next_pass can be compared against overpasses of the site you are interested in using the ESA Orbital Prediction and Overpass Tool (OPOT) at https://evdc.esa.int/orbit/ 


## Getting started
To run the overpass predictor with the given location, run all cells in the notebook starting with the "Load packages" cell.

### Load packages and functions

In [ ]:
import os
import sys
import pandas as pd
import geopandas as gpd
from argparse import Namespace
# Add the parent directory to Python's path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
import next_pass
import opera_products
import plot_maps
import utils
from utils import bbox_type, create_polygon_from_kml
from shapely.geometry import box

# Specify location
Start with selecting the location by  specifying the latitude/longitude (single point or bounding box) or pointing the script to the path of a location file (.kml). Please set location_file_path to '' if using the single point or boundng box options. 

In [ ]:
# Bounding box covering the region impacted by Hurricane Milton
lat_S, lat_N, lon_W, lon_E= (24.5, 32.5, -83.5,-79.5)

### Run next_pass for Sentinel 1/2, and Landsat 8/9
Use next_pass to predict the overpasses of the above satellites over the selected location. The cells below will call next_pass to predict overpasses for the selected satellites, then provide overpasses visualization in an interactive map.

In [ ]:
# SENTINEL-1
args = Namespace(bbox=(lat_S, lat_N, lon_W, lon_E),sat="sentinel-1")
result_s1 = next_pass.find_next_overpass(args)
print("*** SENTINEL 1 ***")
print(result_s1.get("next_collect_info", "No collection info available"))

In [ ]:
# SENTINEL-2
args = Namespace(bbox=(lat_S, lat_N, lon_W, lon_E),sat="sentinel-2")
result_s2 = next_pass.find_next_overpass(args)
print("*** SENTINEL 2 ***")
print(result_s2.get("next_collect_info", "No collection info available"))

In [ ]:
# LANDSAT
args = Namespace(bbox=(lat_S, lat_N, lon_W, lon_E),sat="landsat")
result_l = next_pass.find_next_overpass(args)
print("*** LANDSAT 8/9 ***")
print(result_l.get("next_collect_info", "No collection info available"))

### Overpasses Vizualisation  
The below vizualization tool shows the path of Sentinel 1/2 at the predicted date/time

In [ ]:
m1 = plot_maps.make_overpasses_map(result_s1, result_s2, result_l, args.bbox)
m1

# Available OPERA Products

## Description
* Here we will use Leafmap and Earthaccess to explore OPERA DSWx Products.
* The Leafmap library provides a suite of tools for interactive mapping and visualization in Jupyter Notebooks. Leafmap version 0.30.0 and later offers tools specifically for accessing NASA Earthdata by building on the newly developed NASA Earthaccess library. Earthaccess provides streamlined access to NASA Earthdata and simplifies the authentication and querying process over previously developed approaches. 
* This section is designed to leverage tools within Earthaccess and Leafmap to facilitate easier access and visualization of OPERA data products for a user-specified area of interest (AOI). 

## Display most recent OPERA products at the selected Area of Interest (AOI)
* The script will use the area of interest indicated above (to predict overpasses) to retreive the most recent OPERA products at the time the present notebook is run. We set event_date by default to today, but this can be changed to a prior date if the user is interested in a prior event. 

In [ ]:
args = Namespace(bbox=(lat_S, lat_N, lon_W, lon_E),number_of_dates=5, event_date="today")
results_opera = opera_products.find_print_available_opera_products(args.bbox, args.number_of_dates, args.event_date)
opera_products.export_opera_products(results_opera)

In [ ]:
m2 = plot_maps.make_opera_granule_map(results_opera, args.bbox)
m2

*******************************************************************

## Searching and Visualizing NASA OPERA Data Products Interactively 
* Another option is available through an Earthdata login. You can register for an account at urs.earthdata.nasa.gov. 
* For further information please refer to https://leafmap.org/notebooks/101_nasa_opera/
* Run the Jupyter Notebook until this cell, before using the option below

In [ ]:
import leafmap
leafmap.nasa_data_login()

The map is zoomed in on the defined AOI. Select a dataset from the Short Name dropdown list. Click the "Search" button to load the available datasets for the region. The footprints of the datasets will be displayed on the map. Click on a footprint to display the metadata of the dataset.

In [ ]:
bbox = utils.bbox_type(args.bbox)
if isinstance(args.bbox, str):
    AOI_polygon = create_polygon_from_kml(args.bbox)
    AOI = AOI_polygon.bounds
else:
    lat_min, lat_max, lon_min, lon_max = args.bbox
    AOI = (lon_min, lat_min, lon_max, lat_max)
    AOI_polygon = box(lon_min, lat_min, lon_max, lat_max)

gdf = gpd.GeoDataFrame(geometry=[AOI_polygon], crs="EPSG:4326")
c_lat = (AOI[1] + AOI[3]) / 2
c_lon = (AOI[0] + AOI[2]) / 2

m = leafmap.Map(center=(c_lat, c_lon), zoom=6, height="700px")
m.add_basemap("Satellite")
m.add("NASA_OPERA")
m.add_gdf(gdf, layer_name="AOI", style={"color": "black", "weight": 2, "fillOpacity": 0})

m